# ドメイン特化

ドメイン特化では、モデル呼び出し以前に入力設計と評価設計を固める習慣を作ります。

前提: 確率的な予測モデルの見方と、深層学習の基礎が前提です。

到達目標: プロンプト、事前学習、微調整、RAG、効率化を実装視点で横断し、設計判断を言語化できる状態にします。

ここで扱う中心語は 「トークン」、「事前学習」、「微調整」、「RAG」、「推論最適化」、「ドメイン特化」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは LLM 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 1. トークン近似を体験する

最初に、入力長とトークン量の関係を簡易計測します。コスト管理の第一歩です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
text = '大規模言語モデルは文脈の与え方で応答品質が大きく変わる。'
char_len = len(text)
space_tokens = text.split()
rough_tokens = max(1, char_len // 2)
print('chars=', char_len, 'space_tokens=', len(space_tokens), 'rough_tokens=', rough_tokens)

厳密なトークン化ではありませんが、入力を短く保つ設計感覚を作るには十分です。

たとえば トークン を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 2. プロンプトを構造化する

次に、指示・制約・出力形式を分離したテンプレートを作ります。曖昧さを減らすための実装技法です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
instruction = '勾配降下法を初学者向けに説明する'
constraints = ['120字以内', '比喩は1つまで', '最後に要点を1行でまとめる']
prompt = f"指示: {instruction}\n制約: {'; '.join(constraints)}\n出力:"
print(prompt)
print('prompt_chars=', len(prompt))

この形にすると、失敗原因を特定しやすくなります。品質改善は原因分離のしやすさで決まります。

たとえば トークン を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 式をコードと接続する

ここで、本文中で使っている式を実装視点で再整理します。式を暗記するのではなく、どの変数がどのコード行に対応するかを確認してください。

1. p_theta(x_t | x_<t)
2. L_CE = - sum_t log p_theta(x_t | x_<t)

この一覧は『正解の丸暗記』ではなく、デバッグ時の観点表です。実装が壊れたとき、どの式のどの項が怪しいかを逆算できるように使います。

## 3. 検索文脈を結合する

ここで RAG の最小形を実装します。質問と関連文を結合し、回答入力を作る流れを確認します。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
question = 'ベルマン方程式を高校生向けに説明して'
retrieved = ['価値は将来報酬の割引和で定義する', '現在価値は次状態価値で再帰的に更新できる']
context = '\n'.join(f'- {c}' for c in retrieved)
final_input = f"質問:\n{question}\n\n参考文脈:\n{context}\n\n回答:"
print(final_input)

検索文脈を入れる目的は、モデルの記憶に頼りすぎないことです。根拠付き応答を作りやすくなります。

たとえば トークン を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 4. 評価項目を数値化する

次に、回答を点検するための簡易スコアを定義します。評価軸を言語化すると改善が継続できます。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
answer = 'ベルマン方程式は、今の価値を次の価値で更新する再帰式です。'
checks = {'length_ok': len(answer) <= 120, 'has_keyword': '価値' in answer, 'has_recurrence': '再帰' in answer}
score = sum(1 for v in checks.values() if v) / len(checks)
print('checks=', checks)
print('score=', round(score, 3))

このような軽量評価でも、改善方向を揃える効果があります。実務ではこの評価軸をチームで共有します。

たとえば トークン を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 5. 推論コストを見積もる

最後に、入力と出力の長さから概算コストを計算します。モデル選択は性能だけでなく費用とのバランスが必要です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
input_tokens = 320
output_tokens = 180
price_per_1k = 0.0012
cost = (input_tokens + output_tokens) / 1000 * price_per_1k
print('estimated_cost=', round(cost, 6))

この見積もりを運用前に作ると、スケール時の予算超過を防ぎやすくなります。

たとえば トークン を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## つまずきポイントと確認課題

実装学習では、書けることより説明できることを重視します。次の失敗パターンを先に知っておくと、学習速度が上がります。

1. プロンプトだけで全問題を解決しようとする
2. 評価指標を決めずに改善を繰り返す
3. コストと品質のバランスを見ない

思考実験: 同じ質問に、文脈あり/なしで入力を作り、出力差の理由を分解します。

確認課題
1. このノートのコードのうち1つを選び、行ごとの役割を口頭で説明する。
2. 定数を1つ変更し、出力変化の理由を『入力 -> 中間値 -> 出力』で説明する。
3. 実務利用を想定し、異常入力時に壊れる箇所と防止策を1つ提案する。

次の「軽量化（圧縮・最適化・効率化）」では、同じ確認手順を維持したまま対象を広げます。今回のコードを再実行してから進むと理解がつながります。

補講 1

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 10

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 11

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 12

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 13

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 14

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 15

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 16

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 1

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 10

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 11

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 12

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 13

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 14

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 15

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 16

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 1

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

LLM の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。